In [7]:
#Double lane model overview:
# Similar to the triple lane model, 
#the double lane model includes a right lane where vehicles can either turn right or go straight ahead, 
#and a left lane where vehicles can either turn left or go straight ahead. 
#In order to hold constant the overall traffic flow, 
#the average interarrival time on the right lane and the left lane are ? minutes and ? minutes respectively. 
#The pedestrians’ behavior is assumed to be unchanged. 

# Import all necessary packages
# double-ended queue
from collections import deque 
import numpy as np
import simpy
from simpy.util import start_delayed
#print right turn in red, left turn in blue
from colorama import Fore, Back, Style 

#define a utility class to keep track of the statistics
class Stats(object): 
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

#set a random seed to get reproducible results
random_seed = 0
np.random.seed(random_seed)

#pre-define some useful variables
#total number of minutes to be simulated:
end_time = 60

#cars arrive at the traffic light according to a Poisson process with an
#1 cars per 7.91 seconds
right_interarrival_mean = 15.82/60

p_interarrival_mean = 30/60 #interarrival time for pedestrian set to be 30 seconds

left_interarrival_mean = 15.82/60 #interarrival time for left-turning vehicles set to be 30 seconds

# Traffic light green and red durations:
t_green= 26/60
t_red= 90/60

# The time for a vehicle at the head of the queue to clear the intersection
#(including turn right at red light, turn right at green light, go straight, and turn left) 
#is modeled as four triangular distributions with specified lower bound, mode and upper bound.
#considering normal distribution has negative values which can't be used to model time in simulation,

#turn right at red light
r_t_r_mode = 10/60 
r_t_r_lower = 5/60 
r_t_r_upper = 15/60

#turn right at green light
r_t_mode = 3/60
r_t_lower = 2/60
r_t_upper = 4/60

#go straight 
str_mode = 2/60 
str_lower = 1.8/60
str_upper = 2.2/60

#turn left
l_mode = 30/60
l_lower = 20/60
l_upper = 40/60

#the time for a group of pedestrians to clear the intersection is modeled as a normal distribution
p_crossing_mode = 18/60 
p_crossing_lower = 13/60
p_crossing_upper = 23/60

# Initialize queues for the right-most lane, middle lane, traffic island and for the pedestrians
right_q = deque() 
left_q = deque()
pedestrian = deque()

# initialize the number of vehicles on different lanes and the number of pedestrians
pedestrian_count = r_arrival_count = l_arrival_count = departure_count = 0

#stats for the right lane
r_q_stats = Stats(count=0, cars_waiting=0)
r_w_stats = Stats(count=0, waiting_time=0.0)

#stats for the pedestrians
P_W_stats = Stats(count=0, waiting_time=0.0)
p_num_stats = Stats(count=0, pedestrian_waiting=0)

#stats for the left lane
l_w_stats = Stats(count=0, waiting_time=0)
l_q_stats = Stats(count=0, cars_waiting=0)

def Right_arrival(): #arrival event for the right lane 
    global r_arrival_count, env, light, right_q, right_turn_boolean

    while True:
        
        r_arrival_count+= 1
        right_turn_boolean = np.random.choice([False,True], 1, p=[0.6,0.4])    

        if len(right_q):
            right_q.append((r_arrival_count, env.now))
            print(f"At time {env.now}, vehicle {r_arrival_count} arrived on the right lane and joined the queue at position {len(right_q)}.")
        elif light == 'red':   
            #assume vehicles on the right lane turn right with a 40% chance
            if right_turn_boolean:
                print(Fore.RED + f'At time {env.now}, vehicle {r_arrival_count} arrived and started to turn right.')
                print(Style.RESET_ALL)
                r_w_stats.count += 1
            else:
                right_q.append((r_arrival_count, env.now))
                print(f"At time {env.now}, vehicle {r_arrival_count} arrived on the right lane and joined the queue at position {len(right_q)}.")
        elif right_turn_boolean:
            print(Fore.RED + f'At time {env.now}, vehicle {r_arrival_count} arrived and started to turn right.')
            print(Style.RESET_ALL)
            r_w_stats.count += 1
        else:
            print(f"At time {env.now}, vehicle {r_arrival_count} arrived on the right lane and started to go straight.")
            r_w_stats.count += 1

      # Schedule next arrival:
        yield env.timeout(np.random.exponential(right_interarrival_mean))

def Left_arrival(): #arrival event for the left lane 
    global l_arrival_count, env, light, left_q, left_turn_boolean

    while True:
        
        l_arrival_count+= 1
        left_turn_boolean = np.random.choice([False,True], 1, p=[0.6,0.4])    

        if len(left_q) or light == 'red':   
            left_q.append((l_arrival_count, env.now))
            print(f"At time {env.now}, vehicle {l_arrival_count} arrived on the left lane and joined the queue at position {len(left_q)}.")
        elif left_turn_boolean:
            print(Fore.BLUE + f'At time {env.now}, vehicle {l_arrival_count} arrived and started to turn left.')
            print(Style.RESET_ALL)
            l_w_stats.count += 1
        else:
            print(f"At time {env.now}, vehicle {l_arrival_count} arrived on the left lane and started to go straight.")
            l_w_stats.count += 1

      # Schedule next arrival:
        yield env.timeout(np.random.exponential(left_interarrival_mean))

def Right_departure(): #departure event for the right lane
    global env, right_q

    while True:
        # If the queue is empty, do not schedule the next departure.
        if len(right_q) == 0:
            return
        elif right_turn_boolean:
            if light == 'red':
                car_number, t_arrival= right_q.popleft()
                print(Fore.RED + f"At time {env.now}, {car_number} turned right at red light, leaving {len(right_q)} cars in the queue.")
                print(Style.RESET_ALL)
                r_w_stats.count += 1
                r_w_stats.waiting_time += env.now - t_arrival
                yield env.timeout(np.random.triangular(r_t_r_lower,r_t_r_mode,r_t_r_upper))
            else:
                car_number, t_arrival= right_q.popleft()
                print(Fore.RED +f"At time {env.now}, {car_number} turned right at green light, leaving {len(right_q)} cars in the queue.")
                print(Style.RESET_ALL)
                r_w_stats.count += 1
                r_w_stats.waiting_time += env.now - t_arrival
                yield env.timeout(np.random.triangular(r_t_lower,r_t_mode,r_t_upper))
        elif light == 'red':
            return
        else: #car in queue and car go straight and light is green     
            car_number, t_arrival= right_q.popleft()
            print(f"At time {env.now}, {car_number} departed straight ahead, leaving {len(right_q)} cars in the queue.")
            r_w_stats.count += 1
            r_w_stats.waiting_time += env.now - t_arrival
            yield env.timeout(np.random.triangular(str_lower,str_mode,str_upper))
                  
def Left_departure(): #departure event for the left lane
    global env, left_q

    while True:
        # If the queue is empty or light is red, do not schedule the next departure.
        if len(left_q) == 0 or light == 'red':
            return
        elif left_turn_boolean:
            car_number, t_arrival= left_q.popleft()
            print(Fore.BLUE + f"At time {env.now}, {car_number} turned left, leaving {len(left_q)} cars in the queue.")
            print(Style.RESET_ALL)
            l_w_stats.count += 1
            l_w_stats.waiting_time += env.now - t_arrival
            yield env.timeout(np.random.triangular(l_lower,l_mode,l_upper))
        else: #car in queue and car go straight and light is green     
            car_number, t_arrival= left_q.popleft()
            print(f"At time {env.now}, {car_number} departed straight ahead, leaving {len(left_q)} cars in the queue.")
            l_w_stats.count += 1
            l_w_stats.waiting_time += env.now - t_arrival
            yield env.timeout(np.random.triangular(str_lower,str_mode,str_upper))

def Pedestrian():
    
    global env, light, pedestrian, pedestrian_count
    
    while True:
        
        pedestrian_count += 1
        
        if light == 'red' and len(pedestrian): #assume red light for vehicles means green for pedestrians
            print(f'At time {env.now} pedestrian {pedestrian_count} arrived at the intersection and started to cross the road.')
            P_W_stats.count += 1
        else:
            pedestrian.append((pedestrian_count, env.now))
            print(f'At time{env.now}, pedestrian {pedestrian_count} arrived and started to wait.')
        
        yield env.timeout(np.random.exponential(p_interarrival_mean))
    
def pedestrian_cross():
    global env, pedestrian
    
    while True:
        
        pedestrian_idx, t_arrival = [i[0] for i in pedestrian], [i[1] for i in pedestrian]
        P_W_stats.count += len(pedestrian)
        P_W_stats.waiting_time += len(pedestrian)*env.now - sum(t_arrival)
        pedestrian = deque()
        print(f'Pedestrians {pedestrian_idx} crossed the road at time {env.now}.')
        
        if light == 'green' or len(pedestrian)==0: #assume green light for vehicles means red light for pedestrian
            return
        #schedule next crossing
        yield env.timeout(np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))

def light():
    
    global env, light

    while True:
        
        light = 'green'
        print(Back.GREEN + Style.BRIGHT + f"At time {env.now}, traffic light turned green.")
        print(Style.RESET_ALL)
        
        #right lane
        if len(right_q):
            if right_turn_boolean: #turn right
                start_delayed(env, Right_departure(), delay=np.random.triangular(r_t_lower,r_t_mode,r_t_upper))
            else: #go straight
                start_delayed(env, Right_departure(), delay=np.random.triangular(str_lower,str_mode,str_upper))
        
        #left lane
        if len(left_q):
            if left_turn_boolean: #turn left
                start_delayed(env, Left_departure(), delay=np.random.triangular(l_lower,l_mode,l_upper))
            else: #go straight
                start_delayed(env, Left_departure(), delay=np.random.triangular(str_lower,str_mode,str_upper))
             
        #schedule event that will turn the light red
        yield env.timeout(t_green)

        light = 'red'
        print(Back.RED + Style.BRIGHT +f"At time {env.now}, traffic light turned red.")
        print(Style.RESET_ALL)
        
        if len(pedestrian):
            start_delayed(env,pedestrian_cross(),delay=np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))
        
        if len(right_q) and right_turn_boolean:
            start_delayed(env, Right_departure(), delay=np.random.triangular(r_t_r_lower,r_t_r_mode,r_t_r_upper))

        #schedule event that will turn the light green:
        yield env.timeout(t_red)

#monitor the number of vehicles/pedestrians waiting per minute
def monitor(): 
    global env, r_q_stats, p_num_stats, l_q_stats
    while True:
                  
        r_q_stats.count += 1
        r_q_stats.cars_waiting += len(right_q)
                  
        p_num_stats.count += 1
        p_num_stats.pedestrian_waiting += len(pedestrian)
                  
        l_q_stats.count += 1
        l_q_stats.cars_waiting += len(left_q)
                  
        #check the number of vehicles on each lane and the number of pedestrians every min
        yield env.timeout(1) 


#schedule initial events and run the simulation

print("\nSimulation of vehicles & pedestrians arriving at intersection controlled by traffic light")
print('\nDouble lane model\n\n\n')

#initialize environment:
env= simpy.Environment()

#schedule the first change of the traffic light:
env.process(light())

#schedule first arrival of vehicle on each lane
start_delayed(env, Right_arrival(), delay=np.random.exponential(right_interarrival_mean))
start_delayed(env, Left_arrival(), delay=np.random.exponential(left_interarrival_mean))

#schedule first arrival of pedestrian
start_delayed(env, Pedestrian(), delay=np.random.exponential(p_interarrival_mean))

#schedule first statistical monitoring event:
env.process(monitor())

# Let the simulation run for specified time:
env.run(until=end_time)


# Report statistics.

print(f"\n\n      *** Statistical report(random seed={random_seed}) ***")
print('     ( double-lane with no distracted drivers)\n\n')
print('Right lane stats:')
print(f"Number of vehicles waiting in right lane per minute: {r_q_stats.cars_waiting / float(r_q_stats.count)}")
print(f"Average wait time for a vehicle on the right lane (minutes): {r_w_stats.waiting_time / float(r_w_stats.count)}")
print('\n\nLeft lane stats:')
print(f"Number of vehicles waiting in left lane per minute: {l_q_stats.cars_waiting / float(l_q_stats.count)}")
print(f"Average wait time for a vehicle on left lane (minutes): {l_w_stats.waiting_time / float(l_w_stats.count)}")
print('\n\nPedestrians stats:')
print(f"Number of pedestrians waiting to cross per minute: {p_num_stats.pedestrian_waiting / float(p_num_stats.count)}")
print(f"Average wait time for a pedestrian (minutes): {P_W_stats.waiting_time / float(P_W_stats.count)}")





Simulation of vehicles & pedestrians arriving at intersection controlled by traffic light

Double lane model



At time 0, traffic light turned green.

At time 0.20984557865234, vehicle 1 arrived on the right lane and started to go straight.
At time 0.3311470778353259, vehicle 1 arrived and started to turn left.

At time 0.35513869747118665, vehicle 2 arrived and started to turn right.

At time 0.43333333333333335, traffic light turned red.

At time0.4616115729020344, pedestrian 1 arrived and started to wait.
At time 0.4828923066827906, vehicle 2 arrived on the left lane and joined the queue at position 1.
At time 0.6813473802306044, vehicle 3 arrived on the left lane and joined the queue at position 2.
At time 0.703412623353161 pedestrian 2 arrived at the intersection and started to cross the road.
At time 0.7402553008095849 pedestrian 3 arrived at the intersection and started to cross the road.
At time 0.7858358153690852 pedestrian 4 arrived at the intersection and started to cross 


At time 52.49276116797021, 175 turned right at green light, leaving 3 cars in the queue.

At time 52.52313731322925, vehicle 206 arrived on the left lane and joined the queue at position 16.
At time 52.548127100250504, 176 turned right at green light, leaving 2 cars in the queue.

At time 52.55568216646119, vehicle 179 arrived on the right lane and joined the queue at position 3.
At time 52.60296214261202, 177 turned right at green light, leaving 2 cars in the queue.

At time 52.6333333333333, traffic light turned red.

At time 52.64997084393786, vehicle 180 arrived on the right lane and joined the queue at position 3.
At time 52.655663428479556, vehicle 181 arrived on the right lane and joined the queue at position 4.
At time 52.74677653860117, vehicle 182 arrived on the right lane and joined the queue at position 5.
At time 52.78646557727121, vehicle 183 arrived on the right lane and joined the queue at position 6.
At time 52.80028234297356, vehicle 184 arrived on the right lane and

In [8]:

# Import all necessary packages
# double-ended queue
from collections import deque 
import numpy as np
import simpy
from simpy.util import start_delayed
#print right turn in red, left turn in blue
from colorama import Fore, Back, Style 

#define a utility class to keep track of the statistics
class Stats(object): 
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

#set a random seed to get reproducible results
random_seed = 0
np.random.seed(random_seed)

#pre-define some useful variables
#total number of minutes to be simulated:
end_time = 60

#cars arrive at the traffic light according to a Poisson process with an
#1 cars per 7.91 seconds
right_interarrival_mean = 15.82/60

p_interarrival_mean = 30/60 #interarrival time for pedestrian set to be 30 seconds

left_interarrival_mean = 15.82/60 #interarrival time for left-turning vehicles set to be 30 seconds

# Traffic light green and red durations:
t_green= 26/60
t_red= 90/60

# The time for a vehicle at the head of the queue to clear the intersection
#(including turn right at red light, turn right at green light, go straight, and turn left) 
#is modeled as four triangular distributions with specified lower bound, mode and upper bound.
#considering normal distribution has negative values which can't be used to model time in simulation,

#turn right at red light
r_t_r_mode = 10/60 
r_t_r_lower = 5/60 
r_t_r_upper = 15/60

#turn right at green light
r_t_mode = 3/60
r_t_lower = 2/60
r_t_upper = 4/60

#go straight 
str_mode = 2/60 
str_lower = 1.8/60
str_upper = 2.2/60

#turn left
l_mode = 30/60
l_lower = 20/60
l_upper = 40/60

#the time for a group of pedestrians to clear the intersection is modeled as a normal distribution
p_crossing_mode = 18/60 
p_crossing_lower = 13/60
p_crossing_upper = 23/60

# Initialize queues for the right-most lane, middle lane, traffic island and for the pedestrians
right_q = deque() 
left_q = deque()
pedestrian = deque()

# initialize the number of vehicles on different lanes and the number of pedestrians
pedestrian_count = r_arrival_count = l_arrival_count = departure_count = 0

#stats for the right lane
r_q_stats = Stats(count=0, cars_waiting=0)
r_w_stats = Stats(count=0, waiting_time=0.0)

#stats for the pedestrians
P_W_stats = Stats(count=0, waiting_time=0.0)
p_num_stats = Stats(count=0, pedestrian_waiting=0)

#stats for the left lane
l_w_stats = Stats(count=0, waiting_time=0)
l_q_stats = Stats(count=0, cars_waiting=0)

distracted_p = 0.2

def Right_arrival(): #arrival event for the right lane 
    global r_arrival_count, env, light, right_q, right_turn_boolean, distracted_boolean
    distracted_boolean = np.random.choice([True,False],p=[distracted_p,1-distracted_p])

    while True:
        
        r_arrival_count+= 1
        right_turn_boolean = np.random.choice([False,True], 1, p=[0.6,0.4])    

        if len(right_q):
            right_q.append((r_arrival_count, env.now))
            print(f"At time {env.now}, vehicle {r_arrival_count} arrived on the right lane and joined the queue at position {len(right_q)}.")
        elif light == 'red':   
            #assume vehicles on the right lane turn right with a 40% chance
            if right_turn_boolean:
                print(Fore.RED + f'At time {env.now}, vehicle {r_arrival_count} arrived and started to turn right.')
                print(Style.RESET_ALL)
                r_w_stats.count += 1
            else:
                right_q.append((r_arrival_count, env.now))
                print(f"At time {env.now}, vehicle {r_arrival_count} arrived on the right lane and joined the queue at position {len(right_q)}.")
        elif right_turn_boolean:
            print(Fore.RED + f'At time {env.now}, vehicle {r_arrival_count} arrived and started to turn right.')
            print(Style.RESET_ALL)
            r_w_stats.count += 1
        else:
            print(f"At time {env.now}, vehicle {r_arrival_count} arrived on the right lane and started to go straight.")
            r_w_stats.count += 1

      # Schedule next arrival:
        if distracted_boolean:
            print(Fore.YELLOW + 'Watch out! Distracted driver!')
            print(Style.RESET_ALL)
            yield env.timeout(np.random.exponential(right_interarrival_mean) + 1)
        else:
            yield env.timeout(np.random.exponential(right_interarrival_mean))

def Left_arrival(): #arrival event for the left lane 
    global l_arrival_count, env, light, left_q, left_turn_boolean, distracted_boolean
    distracted_boolean = np.random.choice([True,False],p=[distracted_p,1-distracted_p])

    while True:
        
        l_arrival_count+= 1
        left_turn_boolean = np.random.choice([False,True], 1, p=[0.6,0.4])    

        if len(left_q) or light == 'red':   
            left_q.append((l_arrival_count, env.now))
            print(f"At time {env.now}, vehicle {l_arrival_count} arrived on the left lane and joined the queue at position {len(left_q)}.")
        elif left_turn_boolean:
            print(Fore.BLUE + f'At time {env.now}, vehicle {l_arrival_count} arrived and started to turn left.')
            print(Style.RESET_ALL)
            l_w_stats.count += 1
        else:
            print(f"At time {env.now}, vehicle {l_arrival_count} arrived on the left lane and started to go straight.")
            l_w_stats.count += 1

      # Schedule next arrival:
        if distracted_boolean:
            print(Fore.YELLOW + 'Watch out! Distracted driver!')
            print(Style.RESET_ALL)
            yield env.timeout(np.random.exponential(left_interarrival_mean) + 1)
        else:
            yield env.timeout(np.random.exponential(left_interarrival_mean))

def Right_departure(): #departure event for the right lane
    global env, right_q, distracted_boolean
    distracted_boolean = np.random.choice([True,False],p=[distracted_p,1-distracted_p])

    while True:
        # If the queue is empty, do not schedule the next departure.
        if len(right_q) == 0:
            return
        elif right_turn_boolean:
            if light == 'red':
                car_number, t_arrival= right_q.popleft()
                print(Fore.RED + f"At time {env.now}, {car_number} turned right at red light, leaving {len(right_q)} cars in the queue.")
                print(Style.RESET_ALL)
                r_w_stats.count += 1
                r_w_stats.waiting_time += env.now - t_arrival
                if distracted_boolean:
                    print(Fore.YELLOW + 'Watch out! Distracted driver!')
                    print(Style.RESET_ALL)
                    yield env.timeout(np.random.triangular(r_t_r_lower,r_t_r_mode,r_t_r_upper) + 1)
                else:
                    yield env.timeout(np.random.triangular(r_t_r_lower,r_t_r_mode,r_t_r_upper))
            else:
                car_number, t_arrival= right_q.popleft()
                print(Fore.RED +f"At time {env.now}, {car_number} turned right at green light, leaving {len(right_q)} cars in the queue.")
                print(Style.RESET_ALL)
                r_w_stats.count += 1
                r_w_stats.waiting_time += env.now - t_arrival
                if distracted_boolean:
                    print(Fore.YELLOW + 'Watch out! Distracted driver!')
                    print(Style.RESET_ALL)
                    yield env.timeout(np.random.triangular(r_t_lower,r_t_mode,r_t_upper) + 1)
                else:
                    yield env.timeout(np.random.triangular(r_t_lower,r_t_mode,r_t_upper))
        elif light == 'red':
            return
        else: #car in queue and car go straight and light is green     
            car_number, t_arrival= right_q.popleft()
            print(f"At time {env.now}, {car_number} departed straight ahead, leaving {len(right_q)} cars in the queue.")
            r_w_stats.count += 1
            r_w_stats.waiting_time += env.now - t_arrival
            if distracted_boolean:
                print(Fore.YELLOW + 'Watch out! Distracted driver!')
                print(Style.RESET_ALL)
                yield env.timeout(np.random.triangular(str_lower,str_mode,str_upper) + 1)
            else:
                yield env.timeout(np.random.triangular(str_lower,str_mode,str_upper))
                  
def Left_departure(): #departure event for the left lane
    global env, left_q, distracted_boolean
    distracted_boolean = np.random.choice([True,False],p=[distracted_p,1-distracted_p])

    while True:
        # If the queue is empty or light is red, do not schedule the next departure.
        if len(left_q) == 0 or light == 'red':
            return
        elif left_turn_boolean:
            car_number, t_arrival= left_q.popleft()
            print(Fore.BLUE + f"At time {env.now}, {car_number} turned left, leaving {len(left_q)} cars in the queue.")
            print(Style.RESET_ALL)
            l_w_stats.count += 1
            l_w_stats.waiting_time += env.now - t_arrival
            if distracted_boolean:
                print(Fore.YELLOW + 'Watch out! Distracted driver!')
                print(Style.RESET_ALL)
                yield env.timeout(np.random.triangular(l_lower,l_mode,l_upper) + 1)
            else:
                yield env.timeout(np.random.triangular(l_lower,l_mode,l_upper))
        else: #car in queue and car go straight and light is green     
            car_number, t_arrival= left_q.popleft()
            print(f"At time {env.now}, {car_number} departed straight ahead, leaving {len(left_q)} cars in the queue.")
            l_w_stats.count += 1
            l_w_stats.waiting_time += env.now - t_arrival
            if distracted_boolean:
                print(Fore.YELLOW + 'Watch out! Distracted driver!')
                print(Style.RESET_ALL)
                yield env.timeout(np.random.triangular(str_lower,str_mode,str_upper) + 1)
            else:
                yield env.timeout(np.random.triangular(str_lower,str_mode,str_upper))

def Pedestrian():
    
    global env, light, pedestrian, pedestrian_count
    
    while True:
        
        pedestrian_count += 1
        
        if light == 'red' and len(pedestrian): #assume red light for vehicles means green for pedestrians
            print(f'At time {env.now} pedestrian {pedestrian_count} arrived at the intersection and started to cross the road.')
            P_W_stats.count += 1
        else:
            pedestrian.append((pedestrian_count, env.now))
            print(f'At time{env.now}, pedestrian {pedestrian_count} arrived and started to wait.')
        
        yield env.timeout(np.random.exponential(p_interarrival_mean))
    
def pedestrian_cross():
    global env, pedestrian
    
    while True:
        
        pedestrian_idx, t_arrival = [i[0] for i in pedestrian], [i[1] for i in pedestrian]
        P_W_stats.count += len(pedestrian)
        P_W_stats.waiting_time += len(pedestrian)*env.now - sum(t_arrival)
        pedestrian = deque()
        print(f'Pedestrians {pedestrian_idx} crossed the road at time {env.now}.')
        
        if light == 'green' or len(pedestrian)==0: #assume green light for vehicles means red light for pedestrian
            return
        #schedule next crossing
        yield env.timeout(np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))

def light():
    
    global env, light

    while True:
        
        light = 'green'
        print(Back.GREEN + Style.BRIGHT + f"At time {env.now}, traffic light turned green.")
        print(Style.RESET_ALL)
        
        #right lane
        if len(right_q):
            if right_turn_boolean: #turn right
                if distracted_boolean:
                    start_delayed(env, Right_departure(), delay=np.random.triangular(r_t_lower,r_t_mode,r_t_upper) + 1)
                else:
                    start_delayed(env, Right_departure(), delay=np.random.triangular(r_t_lower,r_t_mode,r_t_upper))
            else: #go straight
                if distracted_boolean:
                    start_delayed(env, Right_departure(), delay=np.random.triangular(str_lower,str_mode,str_upper) + 1)
                else:
                    start_delayed(env, Right_departure(), delay=np.random.triangular(str_lower,str_mode,str_upper))
        
        #left lane
        if len(left_q):
            if left_turn_boolean: #turn left
                if distracted_boolean:
                    start_delayed(env, Left_departure(), delay=np.random.triangular(l_lower,l_mode,l_upper) + 1)
                else:
                    start_delayed(env, Left_departure(), delay=np.random.triangular(l_lower,l_mode,l_upper))
            else: #go straight
                if distracted_boolean:
                    start_delayed(env, Left_departure(), delay=np.random.triangular(str_lower,str_mode,str_upper) + 1)
                else:
                    start_delayed(env, Left_departure(), delay=np.random.triangular(str_lower,str_mode,str_upper))
             
        #schedule event that will turn the light red
        yield env.timeout(t_green)

        light = 'red'
        print(Back.RED + Style.BRIGHT +f"At time {env.now}, traffic light turned red.")
        print(Style.RESET_ALL)
        
        if len(pedestrian):
            start_delayed(env,pedestrian_cross(),delay=np.random.triangular(p_crossing_lower,p_crossing_mode,p_crossing_upper))
        
        if len(right_q) and right_turn_boolean:
            if distracted_boolean:
                start_delayed(env, Right_departure(), delay=np.random.triangular(r_t_r_lower,r_t_r_mode,r_t_r_upper) + 1)
            else:
                start_delayed(env, Right_departure(), delay=np.random.triangular(r_t_r_lower,r_t_r_mode,r_t_r_upper))

        #schedule event that will turn the light green:
        yield env.timeout(t_red)

#monitor the number of vehicles/pedestrians waiting per minute
def monitor(): 
    global env, r_q_stats, p_num_stats, l_q_stats
    while True:
                  
        r_q_stats.count += 1
        r_q_stats.cars_waiting += len(right_q)
                  
        p_num_stats.count += 1
        p_num_stats.pedestrian_waiting += len(pedestrian)
                  
        l_q_stats.count += 1
        l_q_stats.cars_waiting += len(left_q)
                  
        #check the number of vehicles on each lane and the number of pedestrians every min
        yield env.timeout(1) 


#schedule initial events and run the simulation

print("\nSimulation of vehicles & pedestrians arriving at intersection controlled by traffic light")
print('\nDouble lane model\n\n\n')

#initialize environment:
env= simpy.Environment()

#schedule the first change of the traffic light:
env.process(light())

#schedule first arrival of vehicle on each lane
start_delayed(env, Right_arrival(), delay=np.random.exponential(right_interarrival_mean))
start_delayed(env, Left_arrival(), delay=np.random.exponential(left_interarrival_mean))

#schedule first arrival of pedestrian
start_delayed(env, Pedestrian(), delay=np.random.exponential(p_interarrival_mean))

#schedule first statistical monitoring event:
env.process(monitor())

# Let the simulation run for specified time:
env.run(until=end_time)


# Report statistics.

print(f"\n\n      *** Statistical report(random seed={random_seed}) ***")
print(f'     ( double-lane with {distracted_p*100}% distracted drivers)\n\n')
print('Right lane stats:')
print(f"Number of vehicles waiting in right lane per minute: {r_q_stats.cars_waiting / float(r_q_stats.count)}")
print(f"Average wait time for a vehicle on the right lane (minutes): {r_w_stats.waiting_time / float(r_w_stats.count)}")
print('\n\nLeft lane stats:')
print(f"Number of vehicles waiting in left lane per minute: {l_q_stats.cars_waiting / float(l_q_stats.count)}")
print(f"Average wait time for a vehicle on left lane (minutes): {l_w_stats.waiting_time / float(l_w_stats.count)}")
print('\n\nPedestrians stats:')
print(f"Number of pedestrians waiting to cross per minute: {p_num_stats.pedestrian_waiting / float(p_num_stats.count)}")
print(f"Average wait time for a pedestrian (minutes): {P_W_stats.waiting_time / float(P_W_stats.count)}")






Simulation of vehicles & pedestrians arriving at intersection controlled by traffic light

Double lane model



At time 0, traffic light turned green.

At time 0.20984557865234, vehicle 1 arrived on the right lane and started to go straight.
At time 0.3311470778353259, vehicle 1 arrived and started to turn left.

At time 0.43333333333333335, traffic light turned red.

At time0.4616115729020344, pedestrian 1 arrived and started to wait.
At time 0.4835735794191383, vehicle 2 arrived and started to turn right.

At time 0.6820286529669521, vehicle 3 arrived on the right lane and joined the queue at position 1.
At time 0.703412623353161 pedestrian 2 arrived at the intersection and started to cross the road.
At time 0.7402553008095849 pedestrian 3 arrived at the intersection and started to cross the road.
At time 0.7858358153690852 pedestrian 4 arrived at the intersection and started to cross the road.
At time 0.7960486087111976 pedestrian 5 arrived at the intersection and started to cross 